# Class 6: The Bigger Picture

## Guest speaker

### [Riya Singh](https://www.linkedin.com/in/riyasingh)

> Riya Singh is an agile data leader, public health researcher and product manager dedicated to designing, developing and managing context-appropriate data products for low-resource populations. As the Director of Data & Analytics of Dimagi’s US Health Division, Riya oversees technical, product and business strategy of various data offerings and serves as a strategic advisor to their partners. Previously, Riya has led large-scale data technology implementations in the US, and India at local, state and federal levels. Riya has a background in public health research and consulting and studied Cognitive Science and Human Rights at UC Berkeley.

#### Questions?

## [Final Project peer grading](https://python-public-policy.afeld.me/en/columbia/final_project/peer_grading.html)

## Ask Me Anything (AMA)

Have slides on "Python beyond data analysis" as backup, but would rather talk about what you want to hear about.

## Python beyond data analysis

We've been focusing on using Python and pandas for data analysis. What else is Python used for?

### Data engineering

- Automation / recurring processes
- Copying/moving/processing/publishing data, especially Big Data
- Monitoring/alerting

### Web development

- Building web sites that are interactive (more than just content)
- Forms
- Presenting data
- Workflows, such as:
   - Signing up for things
   - Paying for things

### Machine learning

- Statistics, but _fancy_
- Building models
   - [Examples](https://tfhub.dev/)
- Finding patterns
- Recommendations
- Detection

When people say "artificial intelligence," they usually mean "machine learning."

![Diagram showing what type of machine learning may be useful, if at all](https://mitsloan.mit.edu/sites/default/files/2021-04/machine-learning-infographic_2.jpg)

[Source, with more thorough explanation](https://mitsloan.mit.edu/ideas-made-to-matter/machine-learning-explained)

#### The process

_High-level_

1. Create a model
   1. Gather a bunch of data for training
   1. If supervised machine learning, label it (give it the right answers)
   1. Segment into training and test data
   1. Train the model against the training dataset (have it identify patterns)
   1. Test the model against the test dataset
1. Run against new data
1. If reinforcement learning, model refines itself

You have a head start: [The fundamentals](https://python-public-policy.afeld.me/en/columbia/syllabus.html#learning-objectives) are applicable anywhere you're using code.

Thanks to the Reader!

## Thank you!

Keep in touch.